In [1]:
import numpy as np

In [2]:
def convert_polar_to_cartesian(theta: np.ndarray, phi: np.ndarray, radius: float):
    """
    Convert polar coordinates (theta, phi) to Cartesian coordinates (x, y, z).
    """
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    return np.array([x, y, z]).T

In [3]:
def generate_satellite_positions(num_satellites, radius):
    """
    Generate random positions over polar coordinates for a given number of satellites.
    Convert polar coordinates to Cartesian coordinates.
    Ouput shape: (num_satellites, 3)
    """
    theta = np.random.uniform(0, 2 * np.pi, num_satellites)
    phi = np.random.uniform(0, np.pi / 2, num_satellites)  # Only consider the Northern hemisphere
    return convert_polar_to_cartesian(theta, phi, radius)


In [4]:
np.random.seed(42)

sats = generate_satellite_positions(1000, 7000)
sats.shape

(1000, 3)

In [5]:
def choose_start_end(satellite_positions):
    """
    Choose start and end satellites for a transfer operation.
    Ensure that the angle between the two satellites is greater than 135 degrees.
    Return (start_index, end_index).
    """
    n_satellites = satellite_positions.shape[0]
    for i in range(n_satellites):
        for j in range(i + 1, satellite_positions.shape[0]):
            angle = np.arccos(
                np.dot(satellite_positions[i], satellite_positions[j])
                / (np.linalg.norm(satellite_positions[i]) * np.linalg.norm(satellite_positions[j]))
            )
            if angle > 3 * np.pi / 4:  # 135 degrees in radians
                return i, j
    err_msg = (
        "No valid start and end satellites found with angle > 135 degrees. "
        "Consider adjusting satellite positions or the number of satellites."
    )
    raise ValueError(err_msg)

In [6]:
src, dst = choose_start_end(sats)
print(f"Start: {src}, end: {dst}")

Start: 1, end: 954


In [7]:
def find_neighbours(cur_idx, dst_pos, satellite_positions):
    """
    Find all neighbours of the current satellite that satisfy the following conditions:
    - Within the communication range: The distance to the neighbour is less than D_MAX.
    - Forward path: The angles made by the neighbours satellite, the current satellite,
    and the destination satellite are acute.
    Returns N_NEIGHBOURS nearest neighbours.
    """
    # Exclude the current satellite itself
    cur_pos = satellite_positions[cur_idx]  # Get the current coordinates first
    satellite_positions = np.delete(satellite_positions, cur_idx, axis=0)

    # Calculate distances to all other satellites
    neighbour_vectors = satellite_positions - cur_pos
    distances = np.linalg.norm(neighbour_vectors, axis=1)

    # Communication range filter
    within_range = distances < 2000

    # Forward path condition
    dst_vector = dst_pos - cur_pos
    forward = np.dot(neighbour_vectors, dst_vector) > 0.5

    # Combine conditions
    neighbours = np.where(within_range & forward)[0]

    # Sort neighbours by distance to the current satellite
    sorted_indices = np.argsort(distances[neighbours])
    return neighbours[sorted_indices][:10].tolist() if len(neighbours) > 0 else []

In [8]:
dst_pos = sats[dst]

for i in range(500):
    n_neighbours = len(find_neighbours(i, dst_pos, sats))
    if n_neighbours < 10:
        print(f"Satellite {i}, {n_neighbours} neighbours")

Satellite 2, 7 neighbours
Satellite 11, 7 neighbours
Satellite 18, 6 neighbours
Satellite 22, 9 neighbours
Satellite 24, 4 neighbours
Satellite 54, 4 neighbours
Satellite 61, 9 neighbours
Satellite 74, 1 neighbours
Satellite 84, 9 neighbours
Satellite 103, 7 neighbours
Satellite 122, 9 neighbours
Satellite 153, 9 neighbours
Satellite 171, 9 neighbours
Satellite 176, 9 neighbours
Satellite 183, 8 neighbours
Satellite 196, 2 neighbours
Satellite 209, 4 neighbours
Satellite 350, 8 neighbours
Satellite 360, 7 neighbours
Satellite 375, 5 neighbours
Satellite 388, 8 neighbours
Satellite 415, 4 neighbours
Satellite 425, 8 neighbours
Satellite 452, 9 neighbours
Satellite 455, 5 neighbours
Satellite 457, 9 neighbours
Satellite 463, 9 neighbours
Satellite 464, 8 neighbours
Satellite 466, 9 neighbours


In [3]:
from typing import TypedDict, Optional

class Experience(TypedDict):
    """
    A single experience tuple (s, a, r, s').
    Fields can be None if not yet available.
    """

    state: Optional[np.ndarray]
    action: Optional[int]
    reward: Optional[float]
    next_state: Optional[np.ndarray]


class ExperienceBuffer:
    """
    Store incomplete experiences e = (s, a, r, s').
    Necessary because s' IS NOT IMMEDIATELY AVAILABLE after taking action a.
    The next time the same package is processed, we can update s' and push it out of the buffer.
    """

    def __init__(self):
        self.buffer = {}

    def add_experience(self, package_id, experience: Experience):
        """
        Add an experience to the buffer.
        """
        self.buffer[package_id] = experience

    def update_experience(self, package_id, new_experience: Experience):
        """
        Update the state of an experience in the buffer based on package_id.
        """
        if package_id in self.buffer:
            experience = self.buffer[package_id]
            for key in new_experience:
                if new_experience[key] is not None:
                    experience[key] = new_experience[key]
        else:
            raise KeyError(f"Package ID {package_id} not found in buffer.")

    def pop_experience(self, package_id):
        """
        Remove an experience from the buffer based on package_id.
        """
        if package_id in self.buffer:
            return self.buffer.pop(package_id)
        else:
            raise KeyError(f"Package ID {package_id} not found in buffer.")

In [6]:
buffer = ExperienceBuffer()
buffer.add_experience(
    package_id=0,
    experience={"action": 0}
)
buffer.update_experience(
    package_id=0,
    new_experience={"reward": -100}
)
buffer.buffer

{0: {'action': 0, 'reward': -100}}

In [1]:
from typing import List

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df_results = pd.read_csv("data/dqn_results_20251201.csv")
df_results.head()

,AoI,Dropped_Ratio
0,1.3455,0.2048
1,1.2479,0.2229
2,1.2798,0.2892
3,1.2137,0.3253
4,1.2210,0.3855


In [8]:
from src.utils.report import calc_running_mean_adaptive

window_size = 4
aoi_mean = calc_running_mean_adaptive(df_results["AoI"], window_size)
drop_mean = calc_running_mean_adaptive(df_results["Dropped_Ratio"], window_size)

df_mean = pd.DataFrame({
    "aoi_mean": aoi_mean,
    "drop_mean": drop_mean
})

df_mean["fitness"] = df_mean.apply(
    lambda row: ((1 - row["drop_mean"])/row["aoi_mean"]) if row["drop_mean"] < 0.1 else -row["drop_mean"]/row["aoi_mean"],
    axis=1
)
df_mean.tail()

,aoi_mean,drop_mean,fitness
95,0.625650,0.088875,1.456285
96,0.632650,0.102425,-0.161898
97,0.637700,0.131025,-0.205465
98,0.643450,0.138550,-0.215324
99,0.642575,0.126500,-0.196864


In [9]:
max(df_mean["fitness"])

1.4833683190699176

In [ ]:
import cv2
import os
import glob

INPUT_DIR = "figures"
OUTPUT_DIR = "/home/lan/Desktop/figures"

for img_file in glob.glob(os.path.join(os.path.expanduser(INPUT_DIR), "*.png")):
    filename = os.path.basename(img_file)
    print(f"Resizing {filename}...")
    img = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
    h, w = img.shape[:2]
    img_resized = cv2.resize(
    img,
    (int(w * 0.5), int(h * 0.5)),
    interpolation=cv2.INTER_AREA
)
    img_resized.save(f"{OUTPUT_DIR}/{filename}")

Resizing dqn_average_aoi_over_episodes.png...
Resizing dqn_packet_drop_ratio_over_episodes.png...
Resizing average_aoi_comparison.png...
Resizing dqn_objective_function_over_episodes.png...
Resizing objective_function_comparison.png...
Resizing packet_drop_ratio_comparison.png...


In [9]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (67.0 MB)
